<h2>Understanding the Given Data</h2>

In [ ]:
#importing useful libraries
import math
import string
import datetime
import chardet
import fuzzywuzzy
from fuzzywuzzy import process
import warnings
warnings.filterwarnings("ignore")
import os
import scipy
import numpy as np
import pandas as pd
import pandas_profiling as pp
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import plotly.express as px
import plotly.graph_objects as go
sns.color_palette('bright')
sns.set(style='darkgrid',rc = {'figure.figsize':(15,8)})
from plotly.offline import iplot
from sklearn.pipeline import make_pipeline 
%matplotlib inline
print("Ready,set,go....")

In [ ]:
#reading data sets
df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
print(df.shape)

In [ ]:
#checking head
df.head()

In [ ]:
#checking info
df.info()

In [ ]:
#reading data sets
dftest = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
print(dftest.shape)

In [ ]:
dftest.head()

In [ ]:
#checking info
dftest.info()

<h2>visuvalising the  null values</h2>

In [ ]:
#visuvalising the  null values
plt.figure(figsize=(16,10))
sns.heatmap(df.isnull())
plt.title('Heatmap for checking Null Values')
plt.show()

In [ ]:
#visuvalising the  null values
plt.figure(figsize=(16,10))
sns.heatmap(dftest.isnull())
plt.title('Heatmap for checking Null Values')
plt.show()

<h2> understanding the Null values</h2>

In [ ]:
# checking for any NaN values
df.isna().any()

In [ ]:
#getting sum of null values
df.isnull().sum().sort_values(ascending=False)


In [ ]:
# checking for any NaN values
dftest.isna().any()

In [ ]:
#getting sum of null values
dftest.isnull().sum().sort_values(ascending=False)

<h2> Defining useful methods</h2>

In [ ]:
#function for caluclating missing percentage
def missing_percentage(df):
    # get the number of missing data points per column
    missing_values_count = df.isnull().sum()
    total_cells = np.product(df.shape)
    total_missing = missing_values_count.sum()
    # percent of data that is missing
    percent_missing = (total_missing/total_cells) * 100
    print("missing percentage in the given data frame is:{}".format(percent_missing))
    
#function for null dataframe
def null_dataframe(df):
    miss_percent = 100*(df.isnull().sum()/len(df))
    miss_percent = miss_percent[miss_percent>0].sort_values(ascending=False)
    df_null = pd.DataFrame(miss_percent,columns=['percentage'])
    return df_null

#function for plotting graph of null values
def null_graph(df):
    train_null = null_dataframe(df)
    plt.figure(figsize=(16,9))
    sns.barplot(data = train_null,x=train_null.index,y='percentage')
    plt.xticks(rotation = 45)
    plt.show()
    
#function for filling values
def filling_values(df,var,val):
    df[var].fillna(value = val,inplace = True)

In [ ]:
#missing percentage in train
missing_percentage(df)

In [ ]:
#missing percentage in test
missing_percentage(dftest)

In [ ]:
#checking null values percentage
train_null = null_dataframe(df)
train_null

In [ ]:
#graph for null values percentage in train
null_graph(df)

In [ ]:
#checking null values percentage for test
test_null = null_dataframe(dftest)
test_null

In [ ]:
#graph for null values percentage in train
null_graph(dftest)

<h2>Cleaning The Data</h2>

<h4> 1.Train data </h4>

In [ ]:
#dropping unnecessary columns
#dropping values more than 50%
df.drop(['PoolQC','MiscFeature','Alley','Fence','FireplaceQu'],axis=1,inplace=True)
dftest.drop(['PoolQC','MiscFeature','Alley','Fence','FireplaceQu'],axis=1,inplace=True)

In [ ]:
#getting null_dataframe
df_null = null_dataframe(df)
li_train = list(df_null.index)
li_train

In [ ]:
null_graph(df)

In [ ]:
#for getting clear understand about wether none value should include or not
for i in df_null.index:
    if df[i].dtypes == 'object':
        print(i)
        print(list(df[i].unique()))
        print("-"*50)

In [ ]:
#filling the train dataframe
#except lotfrontage 
for i in li_train:
    if (df[i].dtypes!='object') & (i not in ['LotFrontage']):
        filling_values(df,i,0)
    elif (i not in ['LotFrontage']):
        filling_values(df,i,df[i].describe()['top'])

In [ ]:
#checking null graph
null_graph(df)

In [ ]:
#filling lotfrontage with mean
df['LotFrontage'].fillna(value = df['LotFrontage'].mean(),inplace = True)

In [ ]:
missing_percentage(df)

In [ ]:
#visuvalising the  null values
plt.figure(figsize=(16,10))
sns.heatmap(df.isnull())
plt.title('Heatmap for checking Null Values')
plt.show()

<h4>2.Test data</h4>

In [ ]:
#getting null_dataframe
df_null = null_dataframe(dftest)
li_test = list(df_null.index)
li_test

In [ ]:
null_graph(dftest)

In [ ]:
for i in df_null.index:
    if dftest[i].dtypes == 'object':
        print(i)
        print(list(dftest[i].unique()))
        print("-"*50)

In [ ]:
for i in li_test:
    if (dftest[i].dtypes!='object') & (i not in ['LotFrontage']):
        filling_values(dftest,i,0)
    elif (i not in ['LotFrontage']):
        filling_values(dftest,i,dftest[i].describe()['top'])

In [ ]:
null_graph(dftest)

In [ ]:
#filling Lotfrontage with its mean
dftest['LotFrontage'].fillna(value = dftest['LotFrontage'].mean(),inplace = True)

In [ ]:
missing_percentage(dftest)

In [ ]:
#visuvalising the  null values
plt.figure(figsize=(16,10))
sns.heatmap(dftest.isnull())
plt.title('Heatmap for checking Null Values')
plt.show()

<h2> Understanding The Target Vraiable</h2>

In [ ]:
px.bar(df['SalePrice'],template='plotly_dark')

In [ ]:
sns.set(style='darkgrid',rc = {'figure.figsize':(15,8)})
sns.distplot(df['SalePrice'],bins = 30)

<h2>Checking Correlation </h2>

In [ ]:
df_corr = df.corr()
df_corr['SalePrice']

In [ ]:
plt.figure(figsize=(10,8))
plt.plot( df.groupby('YrSold')['SalePrice'].mean())
plt.title("saleprice(mean) trend in every year")
plt.show()

In [ ]:
#checking overallqual because of correlation
sns.set(style='darkgrid',rc = {'figure.figsize':(10,8)})
sns.scatterplot(data = df ,y = 'OverallQual',x = 'SalePrice')

In [ ]:
#checking grlivarea because of correleation
sns.set(style='darkgrid',rc = {'figure.figsize':(10,8)})
sns.scatterplot(data=df,x="GrLivArea",y="SalePrice")

<h2>preparing Training data</h2>

In [ ]:
#columns for encoding

obj_col = df.columns[df.dtypes=='object']

obj_colt = dftest.columns[dftest.dtypes=='object']

In [ ]:
obj_col

In [ ]:
#checking which values have different unique values
for i in obj_col:
    print(i)
    print(list(df[i].unique()))
    print("-"*50)

In [ ]:
obj_colt

In [ ]:
#checking which values have different unique values
for i in obj_colt:
    print(i)
    print(list(dftest[i].unique()))
    print("-"*50)

In [ ]:
df.head()

In [ ]:
dftest.head()

In [ ]:
df= pd.get_dummies(df,columns =["MSZoning","Street","LotShape","LandContour",'LandSlope', 
                                'LotConfig','Neighborhood', 'Condition1', 'BldgType', 
                                'RoofStyle',  'MasVnrType', 'ExterQual', 
                                'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 
                                'BsmtFinType1', 'HeatingQC', 'CentralAir', 
                                'KitchenQual', 'GarageType', 'GarageFinish', 
                                'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition'],drop_first=True)
                   

In [ ]:
df.head()

In [ ]:
dftest= pd.get_dummies(dftest,columns =["MSZoning","Street","LotShape","LandContour",'LandSlope', 
                                'LotConfig','Neighborhood', 'Condition1', 'BldgType', 
                                'RoofStyle',  'MasVnrType', 'ExterQual', 
                                'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 
                                'BsmtFinType1', 'HeatingQC', 'CentralAir', 
                                'KitchenQual', 'GarageType', 'GarageFinish', 
                                'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition'],drop_first=True)

In [ ]:
dftest.head()

In [ ]:
#dropping other values
df.drop(["Exterior2nd","Condition2","BsmtFinType2","Utilities", 'HouseStyle',
         'RoofMatl', 'Exterior1st',  'Heating', 'Electrical', 'Functional', 
        'GarageQual'],axis=1,inplace=True)

In [ ]:
#dropping other values
dftest.drop(["Exterior2nd","Condition2","BsmtFinType2","Utilities", 'HouseStyle',
         'RoofMatl', 'Exterior1st',  'Heating', 'Electrical', 'Functional', 
        'GarageQual'],axis=1,inplace=True)

In [ ]:
#assiging X values
X=df.drop(["SalePrice",'Id'],axis=1)
X = X.values
X.shape

In [ ]:
y = df['SalePrice']
y = y.values
y.shape

In [ ]:
#assiging xtest values
xtest=dftest.drop(['Id'],axis=1)
xtest = xtest.values
xtest.shape

<h2> Selecting Models </h2>

In [ ]:
from sklearn.linear_model import LinearRegression,RidgeCV,LassoCV
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor,XGBRFRegressor

In [ ]:
#train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/3, random_state = 69)

<h2>Linear Regression</h2>

In [ ]:
lr = LinearRegression()
lr.fit(X_train,y_train)

In [ ]:
pred = lr.predict(X_test)

In [ ]:
#evaluvating the regressor performance (near to 1 is good)
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
print(r2_score(y_test,pred)*100)
print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
print('Mean Squred Error:', mean_squared_error(y_test, pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, pred)))

In [ ]:
#visuvalising the errors
sns.set(style='darkgrid',rc = {'figure.figsize':(10,8)})
sns.distplot(pred-y_test,kde=False)

<h2>Ridge Regressor</h2>

In [ ]:
rc = RidgeCV(cv = 20)
rc.fit(X_train,y_train)

In [ ]:
pred = rc.predict(X_test)

In [ ]:
#evaluvating the regressor performance (near to 1 is good)
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
print(r2_score(y_test,pred)*100)
print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
print('Mean Squred Error:', mean_squared_error(y_test, pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, pred)))

In [ ]:
#visuvalising the errors
sns.set(style='darkgrid',rc = {'figure.figsize':(10,8)})
sns.distplot(pred-y_test,kde=False)

<h2>Lasso</h2>

In [ ]:
ls = LassoCV(cv = 20)
ls.fit(X_train,y_train)

In [ ]:
pred = ls.predict(X_test)
#evaluvating the regressor performance (near to 1 is good)
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
print(r2_score(y_test,pred)*100)
print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
print('Mean Squred Error:', mean_squared_error(y_test, pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, pred)))

In [ ]:
#visuvalising the errors
sns.set(style='darkgrid',rc = {'figure.figsize':(10,8)})
sns.distplot(pred-y_test,kde=False)

<h2>Random Forest</h2>

In [ ]:
rf = RandomForestRegressor(max_depth=5,n_estimators = 500)
rf.fit(X_train,y_train)

In [ ]:
pred = rf.predict(xtest)

In [ ]:
pred = rf.predict(X_test)
#evaluvating the regressor performance (near to 1 is good)
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
print(r2_score(y_test,pred)*100)
print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
print('Mean Squred Error:', mean_squared_error(y_test, pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, pred)))

In [ ]:
#visuvalising the errors
sns.set(style='darkgrid',rc = {'figure.figsize':(10,8)})
sns.distplot(pred-y_test,kde=False)

<h2>XGBoost</h2>

In [ ]:
xgb = XGBRegressor()
xgb.fit(X_train,y_train)

In [ ]:
pred = xgb.predict(X_test)
#evaluvating the regressor performance (near to 1 is good)
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
print(r2_score(y_test,pred)*100)
print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
print('Mean Squred Error:', mean_squared_error(y_test, pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, pred)))

In [ ]:
#visuvalising the errors
sns.set(style='darkgrid',rc = {'figure.figsize':(10,8)})
sns.distplot(pred-y_test,kde=False)

In [ ]:
xgb = XGBRFRegressor()
xgb.fit(X_train,y_train)

In [ ]:
pred = xgb.predict(X_test)
#evaluvating the regressor performance (near to 1 is good)
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
print(r2_score(y_test,pred)*100)
print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
print('Mean Squred Error:', mean_squared_error(y_test, pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, pred)))

In [ ]:
#visuvalising the errors
sns.set(style='darkgrid',rc = {'figure.figsize':(10,8)})
sns.distplot(pred-y_test,kde=False)

<h2>GradientBoostingRegressor</h2>

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
gd = GradientBoostingRegressor(n_estimators=300,max_depth=6)
gd.fit(X_train,y_train)

In [ ]:
pred = gd.predict(X_test)
#evaluvating the regressor performance (near to 1 is good)
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
print(r2_score(y_test,pred)*100)
print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
print('Mean Squred Error:', mean_squared_error(y_test, pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, pred)))

In [ ]:
#visuvalising the errors
sns.set(style='darkgrid',rc = {'figure.figsize':(10,8)})
sns.distplot(pred-y_test,kde=False)

<h2> Using Lazypredict </h2

In [ ]:
!pip install lazypredict

In [ ]:
from lazypredict.Supervised import LazyRegressor

In [ ]:
reg = LazyRegressor(verbose=0,ignore_warnings = False,custom_metric=None)

In [ ]:
models,predict = reg.fit(X_train,X_test,y_train,y_test)

In [ ]:
models

GradientBoostingRegressor is winner 